In [ ]:
import os
import cv2
import shutil
import albumentations as A
import random
from glob import glob
from ultralytics import YOLO
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

In [ ]:
class_names = {
    ''
}

In [ ]:
os.makedirs('YOLO/original_img_lab/images', exist_ok=True)

In [ ]:
os.makedirs('YOLO/original_img_lab/labels', exist_ok=True)

In [ ]:
transform = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.ShiftScaleRotate(
            shift_limit=0.02,
            scale_limit=0.1,
            rotate_limit=5,
            border_mode=cv2.BORDER_CONSTANT,
            value=(114, 114, 114),
            p=0.5,
        ),
    ],
    bbox_params=A.BboxParams(
        format="yolo",
        label_fields=["class_labels"],
        min_visibility=0.3,
        check_each_transform=True,
    ),
)

os.makedirs('YOLO/img_lab/images', exist_ok=True)
os.makedirs('YOLO/img_lab/labels', exist_ok=True)

for img_name in os.listdir('YOLO/original_img_lab/images'):
    if not img_name.lower().endswith('.jpg'):
        continue
    base, _ = os.path.splitext(img_name)
    img = cv2.imread(f'YOLO/original_img_lab/images/{img_name}')
    if img is None:
        continue
    label_path = f'YOLO/original_img_lab/labels/{base}.txt'
    with open(label_path) as f:
        labels = [list(map(float, line.split())) for line in f if len(line.split()) == 5]
    bboxes = [x[1:] for x in labels]
    classes = [int(x[0]) for x in labels]
    for i in range(10):
        transformed = transform(image=img, bboxes=bboxes, class_labels=classes)
        if transformed['bboxes']:
            cv2.imwrite(f'YOLO/img_lab/images/{base}_{i}.jpg', transformed['image'])
            with open(f'YOLO/img_lab/labels/{base}_{i}.txt', 'w') as f:
                for (x, y, w, h), c in zip(transformed['bboxes'], transformed['class_labels']):
                    f.write(f"{c} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

In [ ]:
root = Path('YOLO/img_lab')
splits = {'train': 0.8, 'test': 0.1, 'valid': 0.1}

for split in splits:
    (root / split / 'images').mkdir(parents=True, exist_ok=True)
    (root / split / 'labels').mkdir(parents=True, exist_ok=True)

src_images = root / 'images'
src_labels = root / 'labels'

files = [f for f in src_images.iterdir() if f.suffix.lower() in ('.jpg', '.jpeg', '.png')]
random.shuffle(files)

n = len(files)
n_train = int(n * splits['train'])
n_test  = int(n * splits['test'])

train_files = files[:n_train]
test_files  = files[n_train:n_train + n_test]
valid_files = files[n_train + n_test:]

for split, flist in (('train', train_files), ('test', test_files), ('valid', valid_files)):
    for img in flist:
        (root / split / 'images' / img.name).write_bytes(img.read_bytes())
        label_src = src_labels / f'{img.stem}.txt'
        if label_src.exists():
            (root / split / 'labels' / label_src.name).write_bytes(label_src.read_bytes())

In [ ]:
%%writefile data.yaml
train: YOLO/img_lab/train/images
val: YOLO/img_lab/valid/images
test: YOLO/img_lab/test/images

nc: 
names: []

In [ ]:
result = model.train(
    data='data.yaml',
    epochs=100,
    batch=8,
    imgsz=640
)